In [1]:
import notebook_setup

import jax.numpy as jnp
import pandas as pd

from faker import Faker

In [2]:
from src.entities.weekly_patterns import WeeklyPattern, SchedulesWeeklyPattern
from src.entities.data_classes import Shift
from src.entities.shifts import SchedulesShift

fk = Faker()

weekly_patterns = [
    WeeklyPattern(id=f"p{idx}", pattern=p)
    for idx, p in enumerate(
        [
            [True, True, True, True, True, False, False],
            [True, True, True, True, False, False, True],
            [True, True, True, False, False, True, True],
            [True, True, False, False, True, True, True],
            [True, False, False, True, True, True, True],
            [False, False, True, True, True, True, True],
            [False, True, True, True, True, True, False],
            [True, False, True, True, True, True, False],
            [True, True, False, True, True, True, False],
            [True, True, True, False, True, True, False],
            [True, True, False, True, True, False, True],
            [True, False, True, True, True, False, True],
            [False, True, True, True, True, False, True],
            [False, True, True, True, False, True, True],
            [False, True, True, False, True, True, True],
            [True, False, True, True, False, True, True],
        ]
    )
]

shifts = [
    SchedulesShift(
        Shift(id="m1", start_time=7.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="m2", start_time=8.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="m3", start_time=9.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a1", start_time=14.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a2", start_time=15.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a3", start_time=17.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="n", start_time=22.0, duration=9.0, task="base"), type="work"
    ),
]

Develop `_generate_possible_patterns`

In [7]:
off_shift = SchedulesShift(
    Shift(id="off", start_time=0, duration=24.0, task="off"), type="off"
)

blocks_gap: int = 1

single_day_off_start_time_variance: float = 2.0
multi_day_off_start_time_variance: float = 8.0

shifts = shifts
weekly_patterns = weekly_patterns
possible_patterns = []

for weekly_pattern in weekly_patterns:
    pattern = weekly_pattern.pattern
    blocks, edges = weekly_pattern.blocks(blocks_gap)
    if blocks == 1:
        for shift in shifts:
            shift_list = [shift.id if is_work else off_shift.id for is_work in pattern]
            compound_id = f"{weekly_pattern.id}_{shift.id}"
            possible_patterns.append(
                SchedulesWeeklyPattern(
                    id=compound_id, pattern=pattern, shift_list=shift_list
                )
            )

    else:
        for edge in edges:
            consec_off = list.index(pattern[edge:], True)
            for first_shift in shifts:
                for second_shift in shifts:
                    diff_start = abs(second_shift.start_time - first_shift.start_time)
                    if (
                        consec_off == 1
                        and abs(diff_start) > single_day_off_start_time_variance
                    ):
                        print(f"invalid shift combination (1 off)")
                    elif diff_start > multi_day_off_start_time_variance:
                        print(f"invalid shift combination (multi off)")
                    else:
                        shift_list = [
                            *[
                                first_shift.id if is_work else off_shift.id
                                for is_work in pattern[0:edge]
                            ],
                            *[
                                second_shift.id if is_work else off_shift.id
                                for is_work in pattern[edge:]
                            ],
                        ]

                        compound_id = (
                            f"{weekly_pattern.id}_{first_shift.id}_{second_shift.id}"
                        )

                        possible_patterns.append(
                            SchedulesWeeklyPattern(
                                id=compound_id, pattern=pattern, shift_list=shift_list
                            )
                        )

invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift 

In [15]:
import pandas as pd

print(len(possible_patterns))
pd.DataFrame([p.id for p in possible_patterns], columns=["pattern_id"]).sample(
    50
).reset_index(drop=True)

293


,pattern_id
0,p4_m3_m2
1,p2_a1_a1
2,p0_a3
3,p15_m2_m2
4,p10_a2_a3
5,p12_m3_m2
6,p14_m3_m3
7,p4_a2_a1
8,p11_m3_m1
9,p1_a1_a2


Developing ``generate``

Array([1, 0], dtype=int32)

2